# Credit Card Fraud Detection

Due to the large size of dataset, I was unable to include the dataset in github. <br>
Kindly use the following link to download the dataset and extract it. <br>
https://www.kaggle.com/datasets/kartik2112/fraud-detection

Note that the relative path used in pd.read_csv() may vary depending on the folder structure. <br>
Ensure to use the proper relative path for this code to work.

In [106]:
import pandas as pd

from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction import FeatureHasher
from sklearn.model_selection import train_test_split, KFold, cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

## Importing the dataset

In [2]:
df = pd.read_csv("fraudTrain.csv")
df

,Unnamed: 0,trans_date_trans_time,cc_num,merchant,category,amt,first,last,gender,street,...,lat,long,city_pop,job,dob,trans_num,unix_time,merch_lat,merch_long,is_fraud
0,0,2019-01-01 00:00:18,2703186189652095,"fraud_Rippin, Kub and Mann",misc_net,4.97,Jennifer,Banks,F,561 Perry Cove,...,36.0788,-81.1781,3495,"Psychologist, counselling",1988-03-09,0b242abb623afc578575680df30655b9,1325376018,36.011293,-82.048315,0
1,1,2019-01-01 00:00:44,630423337322,"fraud_Heller, Gutmann and Zieme",grocery_pos,107.23,Stephanie,Gill,F,43039 Riley Greens Suite 393,...,48.8878,-118.2105,149,Special educational needs teacher,1978-06-21,1f76529f8574734946361c461b024d99,1325376044,49.159047,-118.186462,0
2,2,2019-01-01 00:00:51,38859492057661,fraud_Lind-Buckridge,entertainment,220.11,Edward,Sanchez,M,594 White Dale Suite 530,...,42.1808,-112.2620,4154,Nature conservation officer,1962-01-19,a1a22d70485983eac12b5b88dad1cf95,1325376051,43.150704,-112.154481,0
3,3,2019-01-01 00:01:16,3534093764340240,"fraud_Kutch, Hermiston and Farrell",gas_transport,45.00,Jeremy,White,M,9443 Cynthia Court Apt. 038,...,46.2306,-112.1138,1939,Patent attorney,1967-01-12,6b849c168bdad6f867558c3793159a81,1325376076,47.034331,-112.561071,0
4,4,2019-01-01 00:03:06,375534208663984,fraud_Keeling-Crist,misc_pos,41.96,Tyler,Garcia,M,408 Bradley Rest,...,38.4207,-79.4629,99,Dance movement psychotherapist,1986-03-28,a41d7549acf90789359a9aa5346dcb46,1325376186,38.674999,-78.632459,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1296670,1296670,2020-06-21 12:12:08,30263540414123,fraud_Reichel Inc,entertainment,15.56,Erik,Patterson,M,162 Jessica Row Apt. 072,...,37.7175,-112.4777,258,Geoscientist,1961-11-24,440b587732da4dc1a6395aba5fb41669,1371816728,36.841266,-111.690765,0
1296671,1296671,2020-06-21 12:12:19,6011149206456997,fraud_Abernathy and Sons,food_dining,51.70,Jeffrey,White,M,8617 Holmes Terrace Suite 651,...,39.2667,-77.5101,100,"Production assistant, television",1979-12-11,278000d2e0d2277d1de2f890067dcc0a,1371816739,38.906881,-78.246528,0
1296672,1296672,2020-06-21 12:12:32,3514865930894695,fraud_Stiedemann Ltd,food_dining,105.93,Christopher,Castaneda,M,1632 Cohen Drive Suite 639,...,32.9396,-105.8189,899,Naval architect,1967-08-30,483f52fe67fabef353d552c1e662974c,1371816752,33.619513,-105.130529,0
1296673,1296673,2020-06-21 12:13:36,2720012583106919,"fraud_Reinger, Weissnat and Strosin",food_dining,74.90,Joseph,Murray,M,42933 Ryan Underpass,...,43.3526,-102.5411,1126,Volunteer coordinator,1980-08-18,d667cdcbadaaed3da3f4020e83591c83,1371816816,42.788940,-103.241160,0


In [3]:
df.shape

(1296675, 23)

In [4]:
df.columns

Index(['Unnamed: 0', 'trans_date_trans_time', 'cc_num', 'merchant', 'category',
       'amt', 'first', 'last', 'gender', 'street', 'city', 'state', 'zip',
       'lat', 'long', 'city_pop', 'job', 'dob', 'trans_num', 'unix_time',
       'merch_lat', 'merch_long', 'is_fraud'],
      dtype='object')

In [5]:
# No. of duplicated entries
df.duplicated().sum()

0

In [6]:
# No. of null entries
df.isnull().sum()

Unnamed: 0               0
trans_date_trans_time    0
cc_num                   0
merchant                 0
category                 0
amt                      0
first                    0
last                     0
gender                   0
street                   0
city                     0
state                    0
zip                      0
lat                      0
long                     0
city_pop                 0
job                      0
dob                      0
trans_num                0
unix_time                0
merch_lat                0
merch_long               0
is_fraud                 0
dtype: int64

## Preprocessing

In [7]:
df.drop("Unnamed: 0", axis=1, inplace=True)

In [8]:
trans_datetime = pd.to_datetime(df["trans_date_trans_time"])

In [9]:
df["day"] = trans_datetime.dt.day
df["month"] = trans_datetime.dt.month
df["year"] = trans_datetime.dt.year

df["hour"] = trans_datetime.dt.hour
df["minute"] = trans_datetime.dt.minute
df["second"] = trans_datetime.dt.second

In [10]:
df.drop("trans_date_trans_time", axis=1, inplace=True)

In [11]:
# Credit card number
# Credit card number cannot directly contribute to Credit Card Fraud
encoder = LabelEncoder()
df["cc_num"] = encoder.fit_transform(df['cc_num'])  # reducing the length of number
df["cc_num"].head()

0    444
1     42
2    237
3    509
4    368
Name: cc_num, dtype: int64

In [12]:
encoder.inverse_transform([42])

array([630423337322], dtype=int64)

## Analysing the attribute "merchant"

This merchant attribute seems to be a complex data - having two informations combined together in a single variable.
<br> Preprocessing this attribute can be challenging. Let's explore and check out!

In [13]:
# All entries of merchant starts with "fraud"
len(df[df["merchant"].str.startswith("fraud") == False])

0

In [14]:
# Is there any repetition?
len(df[df["merchant"].str.startswith("fraud_Rippin")])

3022

In [15]:
# Is fraud_Rippin always with "Kub and Mann"?
len(df[df["merchant"] == "fraud_Rippin, Kub and Mann"])

1267

In [16]:
# So, let's list out the possibilities where "fraud_Rippin" not with "Kub and Mann"
df[(df["merchant"].str.startswith("fraud_Rippin")) & (df["merchant"] != "fraud_Rippin, Kub and Mann")]["merchant"].unique()

array(['fraud_Rippin-VonRueden'], dtype=object)

<br>So, here you go!  Here comes the twist!
"Fraud Rippin" can be with both *Kub and Mann* & *VonRueden* <br>
So, here comes the need to split "fraud_ ..." and other part. <br>
Keeping those two in single variable will make "fraud_Rippin-VonRueden" and "fraud_Rippin, Kub and Mann" totally two different entity. <br>
The "fraud_Rippin" can have his own characteristics, which may reflect in other features. <br>
So, let's split them into two.

<br>Let's collect some unique samples:
- fraud_Rippin, Kub and Mann
- fraud_Lind-Buckridge
- fraud_Herzog Ltd
- fraud_Schmidt and Sons

In [17]:
df["merchant"].unique()[:20]

array(['fraud_Rippin, Kub and Mann', 'fraud_Heller, Gutmann and Zieme',
       'fraud_Lind-Buckridge', 'fraud_Kutch, Hermiston and Farrell',
       'fraud_Keeling-Crist', 'fraud_Stroman, Hudson and Erdman',
       'fraud_Rowe-Vandervort', 'fraud_Corwin-Collins',
       'fraud_Herzog Ltd', 'fraud_Schoen, Kuphal and Nitzsche',
       'fraud_Rutherford-Mertz', 'fraud_Kerluke-Abshire',
       'fraud_Lockman Ltd', 'fraud_Kiehn Inc', 'fraud_Beier-Hyatt',
       'fraud_Schmidt and Sons', 'fraud_Lebsack and Sons',
       'fraud_Mayert Group', 'fraud_Konopelski, Schneider and Hartmann',
       'fraud_Schultz, Simonis and Little'], dtype=object)

In [18]:
# Let's understand the first pattern   "fraud_Rippin, Kub and Mann"
df[(df["merchant"].str.contains(',')) & (df["merchant"].str.contains('-') == False)]["merchant"].str.contains("and")

0          True
1          True
3          True
5          True
9          True
           ... 
1296666    True
1296667    True
1296668    True
1296673    True
1296674    True
Name: merchant, Length: 400966, dtype: bool

In [19]:
# How many of them are true?
df[(df["merchant"].str.contains(',')) & (df["merchant"].str.contains('-') == False)]["merchant"].str.contains("and").sum()

400966

Hence we can conclude that when "," is present, "and" is always present

In [20]:
# presence of ',' with '-'
len(df[(df["merchant"].str.contains(',')) & (df["merchant"].str.contains('-'))])

0

This shows that ',' and '-'  never occured together. <br>
This clearly explains us that pattern 1 and pattern 2 are totally independent and free to proceed. <br>

In [21]:
# Let's analyze the second pattern   "fraud_Lind-Buckridge"
print(df[(df["merchant"].str.contains(" -")) & (df["merchant"].str.contains(",") == False)]["merchant"])
print(df[(df["merchant"].str.contains("- ")) & (df["merchant"].str.contains(",") == False)]["merchant"])

Series([], Name: merchant, dtype: object)
Series([], Name: merchant, dtype: object)


<br>This shows that there's no exceptional spaces just before or after the '-' symbol. Great going!

In [22]:
df[(df["merchant"].str.contains("-")) & (df["merchant"].str.contains(",") == False)]["merchant"].unique()[:5]

array(['fraud_Lind-Buckridge', 'fraud_Keeling-Crist',
       'fraud_Rowe-Vandervort', 'fraud_Corwin-Collins',
       'fraud_Rutherford-Mertz'], dtype=object)

In [23]:
print(df[(df["merchant"].str.contains(" -")) & (df["merchant"].str.contains(",") == False) & (df["merchant"].str.contains('and'))]["merchant"])

Series([], Name: merchant, dtype: object)


<br>When '-' is present, 'and' is never present.
<br>So, clear difference -> if second part contains "and", the word is splitted with ',' else with '-'
<br> Marvellous, let's proceed!

In [24]:
# Pattern 3: fraud_Herzog Ltd
df[(df["merchant"].str.contains(",") == False) & (df["merchant"].str.contains("-") == False)]["merchant"].unique()[:5]

array(['fraud_Herzog Ltd', 'fraud_Lockman Ltd', 'fraud_Kiehn Inc',
       'fraud_Schmidt and Sons', 'fraud_Lebsack and Sons'], dtype=object)

<br>When both the delimiters are absent, it means the merchant name is not mentioned but just the company name is present. 
<br>In that case, the company name may or may not have "and". (this covers pattern 4)
<br>This way, Pattern 3  and  Pattern 4  are under the same umbrella!

*Overall Analysis:*

`if ',' present:` &emsp;&emsp; # Need not check for '-'  (because '-' never present with ',') <br>
`    split with ','` <br><br>
`elif '-' present:` <br>
`    split with '-'` <br><br>
`else:` &emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp; # Neither of them present. <br>
`nothing to split`

## Preprocessing the attribute "merchant"

In [25]:
merchant = []
company = []
for val in df["merchant"]:
    if ',' in val:
        x,y = val.split(',')
        x = x[6:]
    elif '-' in val:
        x,y = val.split("-")
        x = x[6:]
    else:
        x = "Arbitrary" # Not a particular person
        y = val[6:]  
        
    
    merchant.append(x)
    company.append(y)

In [26]:
merchant[:5]

['Rippin', 'Heller', 'Lind', 'Kutch', 'Keeling']

In [27]:
company[:5]

[' Kub and Mann',
 ' Gutmann and Zieme',
 'Buckridge',
 ' Hermiston and Farrell',
 'Crist']

In [28]:
df["merchant_name"] = merchant
df["company_name"] = company

In [29]:
df.drop("merchant", axis=1, inplace=True)

## Preprocessing the attribute "category"

In [30]:
df.columns

Index(['cc_num', 'category', 'amt', 'first', 'last', 'gender', 'street',
       'city', 'state', 'zip', 'lat', 'long', 'city_pop', 'job', 'dob',
       'trans_num', 'unix_time', 'merch_lat', 'merch_long', 'is_fraud', 'day',
       'month', 'year', 'hour', 'minute', 'second', 'merchant_name',
       'company_name'],
      dtype='object')

In [31]:
df["category"].unique()

array(['misc_net', 'grocery_pos', 'entertainment', 'gas_transport',
       'misc_pos', 'grocery_net', 'shopping_net', 'shopping_pos',
       'food_dining', 'personal_care', 'health_fitness', 'travel',
       'kids_pets', 'home'], dtype=object)

<br>Here grocery has two categories pos,net
<br>shopping has two categories pos,net
<br>misc (miscellaneous) has two categories pos,net

- grocery_net and grocery_pos can have some similarities - similarities can be understood only if we separate them. <br>
- Consider cases like "entertainment", "travel", "home"  - it has no subclasses, separating them would lead to null entries

Choice on the trade-off can vary from person-to-person. <br>
Here I choose to split them. Because, according to my interpretation<br>
POS -> Point of Sale -> Credit or debit card (physical transactions) <BR>
NET -> Internet -> Internet banking

Just from this interpretation, we know that NET transactions are more vulnerable to fraud. <br>
So, clear classification between them will help us to know, how much do POS contribute towards fraud and how much does NET.

*Overall approach:* <br>
`if '_' is present: ` <br>
`    split with _` <br>
`else:` <br>
`    no splitting`

In [32]:
trans_category = []
trans_type = []

for val in df["category"]:
    if '_' in val:
        x,y = val.split('_')
    else:
        # This choice of keeping val in x or y can vary from person to person.
        x = val
        y = "others"

    trans_category.append(x)
    trans_type.append(y)

In [33]:
set(trans_category)

{'entertainment',
 'food',
 'gas',
 'grocery',
 'health',
 'home',
 'kids',
 'misc',
 'personal',
 'shopping',
 'travel'}

In [34]:
set(trans_type)

{'care', 'dining', 'fitness', 'net', 'others', 'pets', 'pos', 'transport'}

In [35]:
df["trans_category"] = trans_category
df["trans_type"] = trans_type
df.drop("category", axis=1, inplace=True)

## First name and Last name

In [36]:
len(df["first"].unique())

352

In [37]:
len(df["first"])   # obviously len(df["last"]) would have the same value

1296675

In [38]:
len(df["last"].unique())

481

Usually, name cannot contribute much to the fraud. <br>
But seeing that just 352/481 names repeat so many times to bring more than 12 lakh entries, I'd like to keep them for consideration. <br>
Let's decide on that later

## Analysing the attribute "street"

In [39]:
len(df["street"].unique())

983

In [40]:
# Is "561 Perry Cove" repetitive?
len(df[df["street"] == '561 Perry Cove'])

2028

In [41]:
# Is "Perry Cove" having any other number?
len(df[(df["street"].str.contains('Perry Cove')) & (df["street"] != '561 Perry Cove')])

0

In [42]:
df.head()

,cc_num,amt,first,last,gender,street,city,state,zip,lat,...,day,month,year,hour,minute,second,merchant_name,company_name,trans_category,trans_type
0,444,4.97,Jennifer,Banks,F,561 Perry Cove,Moravian Falls,NC,28654,36.0788,...,1,1,2019,0,0,18,Rippin,Kub and Mann,misc,net
1,42,107.23,Stephanie,Gill,F,43039 Riley Greens Suite 393,Orient,WA,99160,48.8878,...,1,1,2019,0,0,44,Heller,Gutmann and Zieme,grocery,pos
2,237,220.11,Edward,Sanchez,M,594 White Dale Suite 530,Malad City,ID,83252,42.1808,...,1,1,2019,0,0,51,Lind,Buckridge,entertainment,others
3,509,45.00,Jeremy,White,M,9443 Cynthia Court Apt. 038,Boulder,MT,59632,46.2306,...,1,1,2019,0,1,16,Kutch,Hermiston and Farrell,gas,transport
4,368,41.96,Tyler,Garcia,M,408 Bradley Rest,Doe Hill,VA,24433,38.4207,...,1,1,2019,0,3,6,Keeling,Crist,misc,pos


In [43]:
print(len(df[df["street"] == '43039 Riley Greens Suite 393']))
print(df["street"].str.contains('Riley Greens Suite').sum())

3030
3030


In [44]:
print(len(df[df["street"] == '594 White Dale Suite 530']))
print(df["street"].str.contains('White Dale Suite').sum())

503
503


In [45]:
print(len(df[df['street'] == '9443 Cynthia Court Apt. 038']))
print(df["street"].str.contains('Cynthia Court Apt').sum())

493
493


In [46]:
print(len(df[df['street'] == '8617 Holmes Terrace Suite 651']))
print(df['street'].str.contains('Holmes Terrace Suite').sum())

531
531


<br> Seems like those numbers are just representative of the street name itself, and the numbers may not change for a street name. <br>
But checking data like this is a bad idea. We shouldn't conclude anything with just few entries. <br>
Let's run a loop over entire data to check this hypothesis.

In [47]:
# Splitting of street
df["street"].str.split(' ').str.len().unique()

array([3, 5], dtype=int64)

In [48]:
df[df['street'].str.split(' ').str.len() == 3]['street']

0                  561 Perry Cove
4                408 Bradley Rest
5               4655 David Island
11         43576 Kristina Islands
12               3337 Lisa Divide
                    ...          
1296667       97070 Anderson Land
1296668          742 Oneill Shore
1296669           474 Allen Haven
1296673      42933 Ryan Underpass
1296674      135 Joseph Mountains
Name: street, Length: 663417, dtype: object

In [49]:
df[df['street'].str.split(' ').str.len() == 5]['street']

1           43039 Riley Greens Suite 393
2               594 White Dale Suite 530
3            9443 Cynthia Court Apt. 038
6            889 Sarah Station Suite 624
7              231 Flores Pass Suite 720
                       ...              
1296663        852 Thomas Well Suite 339
1296666         6296 John Keys Suite 858
1296670         162 Jessica Row Apt. 072
1296671    8617 Holmes Terrace Suite 651
1296672       1632 Cohen Drive Suite 639
Name: street, Length: 633258, dtype: object

In [50]:
success = True
for i in df['street'].unique():
    st_list = i.split()
    new_list = [x for x in st_list if not x.isdigit()]
    if(len(df[df['street'] == i]) != df['street'].str.contains(' '.join(new_list)).sum()):
        print("Not accepted:", i)
        success=False
if success:
    print("Yes, those numbers always associated with the street name.")

Not accepted: 1499 Michael Rue
Not accepted: 23843 Scott Island
Not accepted: 508 Erin Mount
Not accepted: 611 Michael Rue
Not accepted: 26544 Andrea Glen
Not accepted: 36366 Smith Road
Not accepted: 776 Martin Ridge
Not accepted: 836 Stephanie Union
Not accepted: 478 Alexandria Mount
Not accepted: 3495 Williams Stream


In [51]:
len(df[df["street"].str.contains("Michael Rue")])

3610

In [52]:
len(df[df['street'] == '1499 Michael Rue'])

1535

In [53]:
df[(df['street'].str.contains("Michael Rue"))]['street'].unique()

array(['1499 Michael Rue', '611 Michael Rue'], dtype=object)

<br>Our hypothesis is wrong!! <br>
As we suspected, we have some cases where the numbers vary for same street name, may be it means that the number represents door number. <br>
But since the number of such cases are too low when compared to actual number of entries, I prefer to neglect these. Let's not split!<br>

## Other attributes which need not be preprocessed

In [54]:
df['city'].unique()[:25] # nothing to preprocess

array(['Moravian Falls', 'Orient', 'Malad City', 'Boulder', 'Doe Hill',
       'Dublin', 'Holcomb', 'Edinburg', 'Manor', 'Clarksville',
       'Clarinda', 'Shenandoah Junction', 'Saint Petersburg', 'Grenada',
       'High Rolls Mountain Park', 'Harrington Park', 'Lahoma',
       'Carlisle', 'Harborcreek', 'Elizabeth', 'Methuen', 'Moulton',
       'Plainfield', 'May', 'Waukesha'], dtype=object)

In [55]:
df.state.unique() # nothing to preprocess

array(['NC', 'WA', 'ID', 'MT', 'VA', 'PA', 'KS', 'TN', 'IA', 'WV', 'FL',
       'CA', 'NM', 'NJ', 'OK', 'IN', 'MA', 'TX', 'WI', 'MI', 'WY', 'HI',
       'NE', 'OR', 'LA', 'DC', 'KY', 'NY', 'MS', 'UT', 'AL', 'AR', 'MD',
       'GA', 'ME', 'AZ', 'MN', 'OH', 'CO', 'VT', 'MO', 'SC', 'NV', 'IL',
       'NH', 'SD', 'AK', 'ND', 'CT', 'RI', 'DE'], dtype=object)

In [56]:
df.zip.unique()[:25] # nothing to preprocess

array([28654, 99160, 83252, 59632, 24433, 18917, 67851, 22824, 15665,
       37040, 51632, 25442, 33710, 96038, 88325,  7640, 73754, 47838,
       16421,  7208,  1844, 52572,  7060, 76857, 53186], dtype=int64)

In [57]:
df.lat.unique()[:25] # nothing to preprocess

array([36.0788, 48.8878, 42.1808, 46.2306, 38.4207, 40.375 , 37.9931,
       38.8432, 40.3359, 36.522 , 40.7491, 39.3716, 27.7898, 41.6125,
       32.9396, 40.9918, 36.385 , 38.9763, 42.1767, 40.6747, 42.728 ,
       40.6866, 40.6152, 31.9571, 42.9993])

In [58]:
df.long.unique()[:25] # nothing to preprocess

array([ -81.1781, -118.2105, -112.262 , -112.1138,  -79.4629,  -75.2045,
       -100.9893,  -78.6003,  -79.6607,  -87.349 ,  -95.038 ,  -77.8229,
        -82.7243, -122.5258, -105.8189,  -73.98  ,  -98.0727,  -87.3667,
        -79.9416,  -74.2239,  -71.181 ,  -92.6833,  -74.415 ,  -98.9656,
        -88.2196])

In [59]:
df.city_pop.unique()[:25] # nothing to preprocess

array([  3495,    149,   4154,   1939,     99,   2158,   2691,   6018,
         1472, 151785,   7297,   1925, 341043,    589,    899,   4664,
         1078,   4081,   2518, 124967,  47249,   1132,  71485,   1791,
        95015], dtype=int64)

## Preprocessing the attribute "job"

In [60]:
df.job.unique()[:25]

array(['Psychologist, counselling', 'Special educational needs teacher',
       'Nature conservation officer', 'Patent attorney',
       'Dance movement psychotherapist', 'Transport planner',
       'Arboriculturist', 'Designer, multimedia',
       'Public affairs consultant', 'Pathologist', 'IT trainer',
       'Systems developer', 'Engineer, land', 'Systems analyst',
       'Naval architect', 'Radiographer, diagnostic',
       'Programme researcher, broadcasting/film/video', 'Energy engineer',
       'Event organiser', 'Operational researcher', 'Market researcher',
       'Probation officer', 'Leisure centre manager',
       'Corporate investment banker', 'Therapist, occupational'],
      dtype=object)

In [61]:
df[df.job.str.contains('Psychologist')]['job'].unique()

array(['Psychologist, counselling', 'Psychologist, forensic',
       'Psychologist, sport and exercise', 'Psychologist, clinical'],
      dtype=object)

In [62]:
unique_category = set()
for i in df.job.unique():
    for j in i.split(','):
        unique_category.add(j.strip())

In [63]:
len(unique_category)

526

<br>So, there are 526 unique categories. So, let's do FeatureHashing with 526 features (keeping it close to 526 -> recommended)

In [64]:
hasher = FeatureHasher(n_features=25, input_type='string')
arr = hasher.transform(df.job.str.split(',')).toarray()

In [65]:
df = pd.concat([df,pd.DataFrame(arr)], axis=1)

In [66]:
df.columns = ['cc_num', 'amt', 'first', 'last', 'gender', 'street', 'city',
       'state', 'zip', 'lat', 'long', 'city_pop', 'job', 'dob',
       'trans_num', 'unix_time', 'merch_lat', 'merch_long', 'is_fraud',
       'day', 'month', 'year', 'hour', 'minute', 'second',
       'merchant_name', 'company_name', 'trans_category', 'trans_type'] + ['job'+str(i) for i in range(25)]

In [67]:
df.drop('job', axis=1, inplace=True)

## Other features

In [68]:
len(df.dob.unique())

968

DOB has nothing to do with credit card fraud! <br>

In [69]:
len(df.trans_num.unique()) # nothing to do with credit card fraud

1296675

In [70]:
len(df.unix_time.unique()) # nothing to do with credit card fraud

1274823

In [71]:
len(df.merch_lat.unique()) # may be possible geographically

1247805

In [72]:
len(df.merch_long.unique()) # may be possible geographically

1275745

In [73]:
df.is_fraud.unique() # binary target variable

array([0, 1], dtype=int64)

In [74]:
print(df.day.unique())
print(df.month.unique())
print(df.year.unique())

[ 1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24
 25 26 27 28 29 30 31]
[ 1  2  3  4  5  6  7  8  9 10 11 12]
[2019 2020]


In [75]:
print(df.hour.unique())
print(df.minute.unique())
print(df.second.unique())

[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23]
[ 0  1  3  4  5  6  7  9 10 11 12 13 14 17 18 20 21 22 23 25 26 27 29 30
 31 32 33 34 36 37 39 40 41 42 43 44 45 46 47 48 49 50 52 55 56 57 58 59
  2  8 15 16 19 28 38 53 54 35 51 24]
[18 44 51 16  6  8 42  1 23 53 56 27  3 20 49 58 14 34 37 40  7 15 32 36
 39 41 46 22 45 11 47 10 12 43 48 50 26 38 13 55  9 25 31 57  5 59 17 30
  2 19 52  0 24 21 28 54  4 35 29 33]


In [76]:
df.columns

Index(['cc_num', 'amt', 'first', 'last', 'gender', 'street', 'city', 'state',
       'zip', 'lat', 'long', 'city_pop', 'dob', 'trans_num', 'unix_time',
       'merch_lat', 'merch_long', 'is_fraud', 'day', 'month', 'year', 'hour',
       'minute', 'second', 'merchant_name', 'company_name', 'trans_category',
       'trans_type', 'job0', 'job1', 'job2', 'job3', 'job4', 'job5', 'job6',
       'job7', 'job8', 'job9', 'job10', 'job11', 'job12', 'job13', 'job14',
       'job15', 'job16', 'job17', 'job18', 'job19', 'job20', 'job21', 'job22',
       'job23', 'job24'],
      dtype='object')

## Label Encoding necessary columns

In [77]:
# Dropping the columns which I consider has no clue towards fraud detection
df_new = df.drop(['first', 'last', 'dob', 'trans_num', 'unix_time'], axis=1)
df_new.columns

Index(['cc_num', 'amt', 'gender', 'street', 'city', 'state', 'zip', 'lat',
       'long', 'city_pop', 'merch_lat', 'merch_long', 'is_fraud', 'day',
       'month', 'year', 'hour', 'minute', 'second', 'merchant_name',
       'company_name', 'trans_category', 'trans_type', 'job0', 'job1', 'job2',
       'job3', 'job4', 'job5', 'job6', 'job7', 'job8', 'job9', 'job10',
       'job11', 'job12', 'job13', 'job14', 'job15', 'job16', 'job17', 'job18',
       'job19', 'job20', 'job21', 'job22', 'job23', 'job24'],
      dtype='object')

In [86]:
df_new.head()

,cc_num,amt,gender,street,city,state,zip,lat,long,city_pop,...,job15,job16,job17,job18,job19,job20,job21,job22,job23,job24
0,444,4.97,F,561 Perry Cove,Moravian Falls,NC,28654,36.0788,-81.1781,3495,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,42,107.23,F,43039 Riley Greens Suite 393,Orient,WA,99160,48.8878,-118.2105,149,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-1.0,0.0,0.0
2,237,220.11,M,594 White Dale Suite 530,Malad City,ID,83252,42.1808,-112.2620,4154,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,509,45.00,M,9443 Cynthia Court Apt. 038,Boulder,MT,59632,46.2306,-112.1138,1939,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,368,41.96,M,408 Bradley Rest,Doe Hill,VA,24433,38.4207,-79.4629,99,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [87]:
for col in ['gender', 'street', 'city', 'state', 'zip', 'merchant_name', 'company_name', 'trans_category', 'trans_type']:
    encoder = LabelEncoder()
    df_new[col] = encoder.fit_transform(df_new[col])

In [91]:
df_new.iloc[0] # all became numerical

cc_num             444.000000
amt                  4.970000
gender               0.000000
street             568.000000
city               526.000000
state               27.000000
zip                265.000000
lat                 36.078800
long               -81.178100
city_pop          3495.000000
merch_lat           36.011293
merch_long         -82.048315
is_fraud             0.000000
day                  1.000000
month                1.000000
year              2019.000000
hour                 0.000000
minute               0.000000
second              18.000000
merchant_name      205.000000
company_name       118.000000
trans_category       7.000000
trans_type           3.000000
job0                 0.000000
job1                 0.000000
job2                 0.000000
job3                 0.000000
job4                 0.000000
job5                 1.000000
job6                 0.000000
job7                 0.000000
job8                 0.000000
job9                 0.000000
job10     

In [92]:
X = df_new.drop('is_fraud', axis=1)
y = df_new['is_fraud']

In [100]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [101]:
X_train.shape

(1037340, 47)

In [102]:
y_train.shape

(1037340,)

In [103]:
X_test.shape

(259335, 47)

In [104]:
y_test.shape

(259335,)

## Modeling

In [105]:
lr = LogisticRegression()
lr.fit(X_train, y_train)
y_pred = lr.predict(X_test)
accuracy_score(y_test, y_pred)

C:\Users\User\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


0.993579732778067

In [108]:
# KFold cross validation
lr_kfold = LogisticRegression()
kfold = KFold(n_splits=5, shuffle=True, random_state=42)
cv_scores = cross_val_score(lr_kfold, X, y, cv = kfold)
print(cv_scores)

C:\Users\User\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
C:\Users\User\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stab

[0.99357973 0.99365685 0.99356045 0.99364143 0.99383423]


In [109]:
print("The overall accuracy from 5 KFold cross-validations:",cv_scores.mean())

The overall accuracy from 5 KFold cross-validations: 0.9936545394952475


<br>The consistency in the cross-validation scores proves that the model trained is capable of predicting credit card fraud with remarkable accuracy. <br>
The model is behaving almost the same even when the training data and testing data is changed every time. <br>
Thus, the model to predict the credit card fraud is successfully built. <br>

NOTE: While evaluating the accuracy of the model, the model may take sometime to fit and predict. <br>
This is due to the large dataset we have. This is normal and please wait for it to fit on the training data.